# `canonical_smiles` 对应的`sig_id`数据，然后计算这些基因表达的平均值。
这段代码的主要功能是从 `sig_info` 文件中读取数据，按 `canonical_smiles` 分组，然后并行处理每个 `canonical_smiles` 对应的 `sig_id` 数据，计算基因表达的平均值，并最终将结果保存为 CSV 文件。

1. **读取 `sig_info` 文件**：`read_sig_info` 函数读取 `sig_info` 文件并返回一个 DataFrame。
2. **按 `canonical_smiles` 分组**：`group_by_smiles` 函数按 `canonical_smiles` 分组，返回每个 `canonical_smiles` 对应的所有 `sig_id`。
3. **并行处理**：`extract_and_compute_averages_parallel` 函数使用多进程并行处理每个 `canonical_smiles` 的数据提取和计算任务。
4. **保存结果**：`save_combined_data` 函数将综合的 DataFrame 转置并保存为 CSV 文件。

In [ ]:
import os
import pandas as pd
from cmapPy.pandasGEXpress.parse import parse
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import sys
from tqdm import tqdm  # 导入 tqdm


def read_sig_info(sig_info_path):
    """
    读取 sig_info 文件，并添加 canonical_smiles 列。
    """
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 开始读取 sig_info 文件: {sig_info_path}")
    try:
        sig_info = pd.read_csv(sig_info_path, sep="\t")
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 成功读取 sig_info 文件，共包含 {len(sig_info)} 条记录。")
        return sig_info
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 读取 sig_info 文件时出错: {e}", file=sys.stderr)
        return None


def group_by_smiles(sig_info_df):
    """
    按照 canonical_smiles 分组，获取每个 canonical_smiles 对应的所有 sig_id。
    """
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 开始按 'canonical_smiles' 分组.")
    if 'canonical_smiles' not in sig_info_df.columns:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 错误: sig_info 文件缺少 'canonical_smiles' 列。", file=sys.stderr)
        return None

    grouped = sig_info_df.groupby('canonical_smiles')['sig_id'].apply(list).to_dict()
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 共分组 {len(grouped)} 个不同的 'canonical_smiles'。")
    return grouped


def process_smiles(smiles, sig_ids, gctx_path):
    """
    处理单个 canonical_smiles 的数据提取和计算任务。
    """
    start_time = time.time()
    try:
        # 提取对应的 sig_id 数据
        gctx_data = parse(gctx_path, cid=sig_ids)
        data_df = gctx_data.data_df  # 行：基因，列：sig_id

        # 计算每行（基因）的平均值
        gene_means = data_df.mean(axis=1)
        gene_means.name = smiles  # 设置 Series 名称为 canonical_smiles

        # 打印一些统计信息
        elapsed = time.time() - start_time
        print(
            f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 完成 'canonical_smiles' {smiles} 的处理。"
            f" 数据维度: {data_df.shape}，耗时: {elapsed:.2f} 秒。"
        )
        return gene_means

    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 错误: 处理 'canonical_smiles' {smiles} 时出错: {e}", file=sys.stderr)
        return None


def extract_and_compute_averages_parallel(gctx_path, grouped_data, max_workers=4):
    """
    使用并行处理提取和计算平均值，并一次性合并所有结果。
    集成 tqdm 进度条以显示处理进度。
    """
    combined_df = None
    total_smiles = len(grouped_data)
    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 开始并行提取表达数据并计算平均值，共 {total_smiles} 个 'canonical_smiles'。")

    gene_means_list = []  # 用于收集所有 gene_means
    smiles_list = []      # 用于收集对应的 canonical_smiles

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(process_smiles, smiles, sig_ids, gctx_path): smiles
            for smiles, sig_ids in grouped_data.items()
        }

        # 使用 tqdm 包装 as_completed 生成器
        for future in tqdm(as_completed(futures), total=total_smiles, desc="Processing smiles"):
            smiles = futures[future]
            try:
                result = future.result()
                if result is not None:
                    gene_means_list.append(result)
                    smiles_list.append(smiles)
            except Exception as e:
                print(
                    f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 错误: 并行任务处理 'canonical_smiles' {smiles} 时出错: {e}",
                    file=sys.stderr,
                )

    if gene_means_list:
        # 将所有 gene_means 合并为一个 DataFrame
        combined_df = pd.concat(gene_means_list, axis=1)
        combined_df.columns = smiles_list  # 替换列名为 canonical_smiles
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 并行处理完成。")
        print(f"    综合 DataFrame 的维度: {combined_df.shape}")
    else:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 没有有效的 gene_means 结果。", file=sys.stderr)

    return combined_df


def save_combined_data(combined_df, output_path):
    """
    将综合 DataFrame 保存为 CSV 文件，并转置。
    """
    print(f"\n[{time.strftime('%Y-%m-%d %H:%M:%S')}] 开始保存综合数据到 CSV 文件: {output_path}")
    try:
        # 转置 DataFrame，使 canonical_smiles 成为行，基因 ID 成为列
        combined_df = combined_df.T
        # 重置索引，将 canonical_smiles 作为一列
        combined_df.reset_index(inplace=True)
        # 重命名列名
        combined_df.rename(columns={'index': 'smiles'}, inplace=True)
        # 保存为 CSV 文件，不保存索引
        combined_df.to_csv(output_path, index=False)
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 综合数据已成功保存至: {output_path}")
    except Exception as e:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 保存综合数据时出错: {e}", file=sys.stderr)


def main():
    # 设置文件路径
    sig_info_path = "/mnt/d/Research/PHD/DLEPS/results/GSE92742_Broad_LINCS_sig_info_final.txt" # 上一个1.BRD ID_TO_SMILES.ipynb代码输出的文件
    gctx_path = "/mnt/d/Research/PHD/DLEPS/data/GSE92742/GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx"
    output_combined_path = "/mnt/d/Research/PHD/DLEPS/results/combined_smiles_averages.csv"

    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 脚本开始运行.")
    print(f"    sig_info 文件路径: {sig_info_path}")
    print(f"    .gctx 文件路径: {gctx_path}")
    print(f"    输出文件路径: {output_combined_path}")

    # 1. 读取 sig_info 文件
    sig_info_df = read_sig_info(sig_info_path)
    if sig_info_df is None:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 无法读取 sig_info 文件，脚本终止。", file=sys.stderr)
        return

    # 2. 按 canonical_smiles 分组
    grouped_data = group_by_smiles(sig_info_df)
    if grouped_data is None:
        return

    # 3. 使用并行提取表达数据并计算平均值
    combined_df = extract_and_compute_averages_parallel(gctx_path, grouped_data, max_workers=32)  # 根据CPU核数调整 max_workers

    if combined_df is not None and not combined_df.empty:
        # 4. 保存综合数据
        save_combined_data(combined_df, output_combined_path)
    else:
        print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 没有数据可保存。", file=sys.stderr)

    print(f"[{time.strftime('%Y-%m-%d %H:%M:%S')}] 脚本运行结束.")


if __name__ == "__main__":
    main()


# 输出文件数据分析

In [ ]:
import pandas as pd

# 读取文件，尝试不同的分隔符
file_path = '/mnt/d/Research/PHD/DLEPS/results/combined_smiles_averages.csv'
try:
    df = pd.read_csv(file_path, sep=',')
except Exception as e:
    print(f"读取文件时出错: {e}")
    df = pd.read_csv(file_path)  # 尝试默认分隔符

# 打印列名
print("列名:", df.columns.tolist())

# # 查看前几行
# print(df.head())
# # 查看后几行
# print(df.tail())

# 检查'smiles'列是否存在
if 'smiles' in df.columns:
    unique_smiles = df['smiles'].nunique()
    print(f'Unique SMILES: {unique_smiles}')
else:
    print("错误: 'smiles' 列不存在于DataFrame中。请检查列名是否正确。")

# 检查数据结构
if df is not None:
    # 获取行数和列数
    num_rows, num_cols = df.shape
    print(f"数据集包含 {num_rows} 行和 {num_cols} 列。")

# 只保留landmark基因的列数据
1. 从文件 `/mnt/d/Research/PHD/DLEPS/results/combined_smiles_averages.csv` 中加载数据。

2. 从文件 `/mnt/d/Research/PHD/DLEPS/data/GSE92742/GSE92742_Broad_LINCS_gene_info_delta_landmark.txt` 中提取 `landmark` 基因的 ID。

3. 只保留 `landmark` 基因对应的列，输出结果到`/mnt/d/Research/PHD/DLEPS/results/filtered_landmark_smiles.csv`。

In [ ]:
import pandas as pd

# 文件路径
combined_smiles_path = '/mnt/d/Research/PHD/DLEPS/results/combined_smiles_averages.csv'
landmark_genes_path = '/mnt/d/Research/PHD/DLEPS/data/GSE92742/GSE92742_Broad_LINCS_gene_info.txt'

# 1. 读取 combined_smiles_averages.csv 文件
try:
    combined_df = pd.read_csv(combined_smiles_path, sep=',')
    print(f"成功读取文件: {combined_smiles_path}")
except Exception as e:
    print(f"读取文件时出错: {e}")
    combined_df = None

if combined_df is not None:
    # 查看前几行以确认加载成功
    print("combined_smiles_averages.csv 文件前几行:")
    print(combined_df.head())

    # 2. 读取 landmark 基因列表
    try:
        landmark_df = pd.read_csv(landmark_genes_path, sep='\t')
        print(f"成功读取文件: {landmark_genes_path}")
    except Exception as e:
        print(f"读取 landmark 基因文件时出错: {e}")
        landmark_df = None

    if landmark_df is not None:
        # 筛选 pr_is_lm=1 的基因
        landmark_genes = landmark_df[landmark_df['pr_is_lm'] == 1]['pr_gene_id'].astype(str).tolist()
        print(f"landmark 基因数量 (pr_is_lm=1): {len(landmark_genes)}")

        # 检查 landmark 基因列是否在 combined_smiles_averages.csv 中
        available_genes = [gene for gene in landmark_genes if gene in combined_df.columns]
        print(f"在数据中找到的 landmark 基因数量: {len(available_genes)}")

        # 找到未匹配的基因
        missing_genes = [gene for gene in landmark_genes if gene not in combined_df.columns]

        # 打印未找到的基因
        if missing_genes:
            print(f"未找到的 landmark 基因数量: {len(missing_genes)}")
            print("未找到的基因列表:")
            print(missing_genes)
        else:
            print("所有 landmark 基因都在数据中找到了！")

        # 只保留 'smiles' 列和 landmark 基因对应的列
        columns_to_keep = ['smiles'] + available_genes
        filtered_df = combined_df[columns_to_keep]

        # 输出结果
        print("过滤后的 DataFrame:")
        print(filtered_df.head())

        # 保存过滤后的 DataFrame
        output_filtered_path = '/mnt/d/Research/PHD/DLEPS/results/L1000_landmark.csv'
        filtered_df.to_csv(output_filtered_path, index=False)
        print(f"过滤后的数据已保存至: {output_filtered_path}")


# 数据结构

In [ ]:
import pandas as pd

# 文件路径
filtered_landmark_path = '/mnt/d/Research/PHD/DLEPS/results/L1000_landmark.csv'

# 读取数据
try:
    filtered_df = pd.read_csv(filtered_landmark_path)
    print(f"成功读取文件: {filtered_landmark_path}")
except Exception as e:
    print(f"读取文件时出错: {e}")
    filtered_df = None

# 检查数据结构
if filtered_df is not None:
    # 获取行数和列数
    num_rows, num_cols = filtered_df.shape
    print(f"数据集包含 {num_rows} 行和 {num_cols} 列。")


# 将 `/mnt/d/Research/PHD/DLEPS/results/filtered_landmark_smiles.csv` 文件的 smiles 列划分为训练集和测试集

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 文件路径
filtered_landmark_path = '/mnt/d/Research/PHD/DLEPS/results/L1000_landmark.csv'
train_output_path = '/mnt/d/Research/PHD/DLEPS/results/train_SMILES_demo.csv'
test_output_path = '/mnt/d/Research/PHD/DLEPS/results/test_SMILES_demo.csv'

# 读取数据
try:
    filtered_df = pd.read_csv(filtered_landmark_path)
    print(f"成功读取文件: {filtered_landmark_path}")
except Exception as e:
    print(f"读取文件时出错: {e}")
    filtered_df = None

# 提取 SMILES 列并划分数据集
if filtered_df is not None:
    if 'smiles' in filtered_df.columns:
        # 提取 smiles 列
        smiles = filtered_df['smiles']
        print(f"成功提取 'smiles' 列，包含 {len(smiles)} 条记录。")

        # 划分训练集和测试集
        train_smiles, test_smiles = train_test_split(smiles, test_size=0.2, random_state=42)
        print(f"划分完成：训练集 {len(train_smiles)} 条，测试集 {len(test_smiles)} 条。")

        # 保存训练集和测试集
        try:
            train_smiles.to_csv(train_output_path, index=False, header=['smiles'])
            test_smiles.to_csv(test_output_path, index=False, header=['smiles'])
            print(f"训练集已保存至: {train_output_path}")
            print(f"测试集已保存至: {test_output_path}")
        except Exception as e:
            print(f"保存文件时出错: {e}")
    else:
        print("错误: 数据集中不存在 'smiles' 列。")
